In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import sklearn.preprocessing as preprocessing
import os

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings(action='ignore')
from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [2]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions import inference, fitness_fn, generation_valid, generation_test, calculate_fitness

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution




In [3]:
import kernel_fn

ModuleNotFoundError: No module named 'kernel_fn'

In [3]:
torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

prescriptor = Prescriptor(
    basic_block=None, 
    base_small_input_dim=19, 
    base_large_input_dim=19,
    base_hidden_dim=24, 
    base_output_dim=16, 
    after_input_dim=19, 
    after_hidden_dim=32, 
    after_output_dim=6, 
    num_blocks=10000,
).eval()

state_dict_path = '/root/daily/bit/generation/generation_6.pt'
if os.path.exists(state_dict_path):
    state_dict = torch.load(state_dict_path)
    start_gen_idx = state_dict['generation'] + 1
    best_profit = state_dict['best_profit']
    best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

In [4]:
optimal_solution = best_profit[(best_profit[:, 6] > 1.2) & (best_profit[:, 4] > 0.5)]
optimal_index = torch.where((best_profit[:, 6] > 1.2) & (best_profit[:, 4] > 0.5))[0]

In [6]:
optimal_index

tensor([ 1, 18, 29, 35, 36, 46, 57, 77, 86, 88])

In [8]:
best_profit[optimal_index][:, 6]

tensor([1.2311, 1.5522, 1.7069, 3.2987, 1.4191, 2.1319, 1.2713, 1.2728, 1.5994,
        1.6116])

In [9]:
fitness = calculate_fitness(deepcopy(optimal_solution.numpy()))
fitness_index = np.argsort(-fitness)

In [35]:
index = 1

In [36]:
optimal_solution[fitness_index][:, 0][index]

tensor(0.8780)

In [37]:
optimal_solution[fitness_index][:, 1][index]

tensor(0.1915)

In [38]:
optimal_solution[fitness_index][:, 2][index]

tensor(0.1259)

In [39]:
optimal_solution[fitness_index][:, 3][index]

tensor(1.8843)

In [40]:
optimal_solution[fitness_index][:, 4][index]

tensor(0.8261)

In [41]:
optimal_solution[fitness_index][:, 5][index]

tensor(20.8242)

In [42]:
optimal_solution[fitness_index][:, 6][index]

tensor(1.4191)